In [2]:
import numpy as np
import pandas as pd
import os
import  chardet
print(os.getcwd())
import matplotlib.pyplot as plt

distributions = ['DIST_long', 'DIST_short']
leagues = ['ENG1','GER1']
seasons = ['2017_2018','2018_2019']


def plot_rps(x_axis, y_axis, title, x_label, y_label, saving_location, fname):
    # plt.
    plt.plot(x_axis, y_axis, color ='maroon')
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.axhline(y=np.mean(np.array(y_axis)), color ='blue')

    fig = plt.gcf()
    fig.set_size_inches(6, 6)
    export_directory= f'Export\\Images\\{saving_location}'
    if not os.path.exists(export_directory):
        os.makedirs(export_directory)
    plt.savefig(f'{export_directory}\\{fname}',dpi=300, bbox_inches='tight')
    plt.close(fig)
    # plt.show()

def plot_points(x_axis, y_axis, title, x_label, y_label, saving_location, fname):
    # plt.

    y_axis = np.cumsum(np.array(y_axis), axis=0) / np.array(np.arange(1,len(y_axis)+1))
    plt.plot(x_axis, y_axis, color ='maroon')
    plt.title(title)

    plt.xlabel(x_label)
    plt.ylabel(y_label)
    fig = plt.gcf()
    fig.set_size_inches(6, 6)

    fig.set_size_inches(6, 6)
    export_directory = f'Export\\Images\\{saving_location}'

    if not os.path.exists(export_directory):
        os.makedirs(export_directory)
    plt.savefig(f'{export_directory}\\{fname}', dpi=300, bbox_inches='tight')
    plt.close(fig)
    # plt.show()

def get_max_line(x):

    y = np.zeros(shape=[np.shape(x)[1]])
    for i in range(np.shape(x)[1]):
        arr = x[:,i]
        y[i] += max(arr)
    return y.tolist()

def get_min_line(x):

    y = np.zeros(shape=[np.shape(x)[1]])
    for i in range(np.shape(x)[1]):
        arr = x[:,i]
        y[i] += min(arr)
    return y.tolist()

def get_avg_line(x):
    # print(np.shape(x)[0],np.shape(x)[1])
    y = np.zeros(shape=[np.shape(x)[1]])
    for z in x.tolist():
        y += np.array(z)
    # print(y)
    # print(np.shape(y))
    y /= np.shape(x)[0]
    return y.tolist()

def plot_balances_simulated(league_bal,x_axis, y_axis, title, x_label, y_label, saving_location,fname):
    # plt.
    for y_list in y_axis:
        color = ""
        last_bal = y_list[-1]
        if last_bal < league_bal:
            # color += "red"
            color += "lightcoral"
        elif last_bal > league_bal:
            # color += "green"
            color += "lightgreen"
        else:
            # sum_y == 1000
            color += "blue"
        plt.plot(x_axis, y_list, color=color)

    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.plot(x_axis, get_max_line(y_axis), color ='black')
    # plt.text(x=400, y=get_max_line(y_axis)[-1]+2, s='Maximum',color='black', ha='left', va='center', fontsize=12)
    fig = plt.gcf()
    plt.plot(x_axis, get_min_line(y_axis), color ='black')
    # plt.text(x=400, y=get_min_line(y_axis)[-1]+2, s='Minimum',color='black', ha='left', va='center', fontsize=12)
    fig = plt.gcf()
    plt.plot(x_axis, get_avg_line(y_axis), color ='black')
    # plt.text(x=400, y=get_avg_line(y_axis)[-1]+2, s='Average',color='black', ha='left', va='center', fontsize=12)
    fig = plt.gcf()
    fig.set_size_inches(6, 6)
    export_directory = f'Export\\Images\\{saving_location}'
    if not os.path.exists(export_directory):
        os.makedirs(export_directory)
    plt.savefig(f'{export_directory}\\{fname}', dpi=300, bbox_inches='tight')
    plt.close(fig)
    # plt.show()

def accuracy_indicator_function():
    max_indices_f_prob = np.argmax(f_prob, axis=1) # get indices of maximum
    max_indices_o_true = np.argmax(o_true, axis=1) # get indices of maximum
    # xor = np.logical_xor(max_indices_f_prob, max_indices_o_true)
    # not_xor = np.logical_not(xor).astype(int)
    # return not_xor
    list = []
    for i in range(0,len(max_indices_o_true)):
        if max_indices_f_prob[i] == max_indices_o_true[i]:
            list.append(1)
        else:
            list.append(0)
    return np.array(list)

import re


experiments = ['10 Window Experiments','15 Window Experiments','20 Window Experiments','ELO Model Experiments']
tournament_col = []
export_images = True
export_files = False
for experiment in experiments:
    print(experiment)

    dist_col = []
    league_col = []
    rps_col = []
    brier_col = []
    acc_col = []
    simple_col = []
    kelly_col = []
    data_eng_tournament = {}
    data_ger_tournament = {}
    # i = 0
    # j=0
    experiment_id = 0
    for league in leagues:
        for dist in distributions:
            for season in seasons:
                experiment_id += 1
                print(f"Exporting Experiment {experiment_id}")
                # keywords = f'{dist}&{league}&{season}'
                filename = ""
                directory =f"./{experiment}"
                # print(os.listdir(directory))
                for name in os.listdir(directory):
                # load the data from the .npz file
                #     print(name)
                    dist_index = name.find(dist)
                    league_index = name.find(league)
                    season_index = name.find(season)
                    # if re.search(keywords,name):
                    #     print(keywords, "found", name)
                    # print(dist_index,league_index,season_index)
                    if dist_index > 0 and league_index > 0 and season_index > 0:
                        filename = f"{directory}/{name}"
                        loaded_data = np.load(filename, allow_pickle=True)

                        # Extract the arrays from the loaded data
                        team_names = loaded_data["team_names"]
                        expected_points = loaded_data["expected_points"]
                        all_points = loaded_data["all_points"]
                        # current_points = loaded_data["c"]
                        f_HDA_array = loaded_data["f_HDA"]
                        f_HDA_year_range_array = loaded_data["f_HDA_year_range"]
                        f_HDA_metadata_array = loaded_data["f_HDA_metadata"]
                        loaded_score_log = loaded_data["score_log"]
                        YYYY_range_file_names = loaded_data["YYYY_range_file_names"]
                        frequencies_home_win = loaded_data["frequencies_home_win"]
                        frequencies_home_draw = loaded_data["frequencies_home_draw"]
                        frequencies_home_loss = loaded_data["frequencies_home_loss"]
                        simulated_simple_balance = loaded_data["simulated_simple_balance"]
                        simulated_kelly_balance = loaded_data["simulated_kelly_balance"]
                        brier_score_and_rps_avg = loaded_data["brier_score_and_rps_avg"]
                        rps_all = loaded_data["rps_all"]
                        o_true = loaded_data["o_true"]
                        f_prob = loaded_data["f_prob"]
                        country_data = loaded_data["country_data"]
                        level_data = loaded_data["level_data"]

                        # get original dictionaries
                        f_HDA = { 'H': f_HDA_array[0], 'D': f_HDA_array[1], 'A': f_HDA_array[2] }
                        f_HDA_year_range = { 'YYYY_start': f_HDA_year_range_array[0], 'YYYY_end': f_HDA_year_range_array[1] }
                        f_HDA_metadata = { 'Countries': f_HDA_metadata_array[0], 'Level': f_HDA_metadata_array[1] }

                        # get ordered points dataframe
                        team_names = team_names.tolist()
                        expected_points = expected_points.tolist()
                        points_df = pd.DataFrame(np.array([team_names,expected_points]).T, columns=["Team","Simulated Average Points"])
                        points_df = points_df.sort_values(by=["Simulated Average Points"], ascending=False, ignore_index=True)
                        points_df.index += 1
                        brier_score = brier_score_and_rps_avg[0]

                        rps_avg = brier_score_and_rps_avg[1]
                        Country = country_data[0]
                        Level = level_data[0]
                        YYYY_start = YYYY_range_file_names[0]
                        YYYY_end = YYYY_range_file_names[1]
                        k = len(simulated_simple_balance)

                        indicator_list = accuracy_indicator_function()
                        accuracy = np.sum(indicator_list)/len(indicator_list)


                        all_points = all_points.tolist()

                        mean_balance_simple = (np.mean(np.array(simulated_simple_balance[:,[-1]])))
                        mean_balance_kelly = (np.mean(np.array(simulated_kelly_balance[:,[-1]])))
                        initial_balance = len(f_prob) * 10
                        # print(initial_balance)
                        profit_simple = mean_balance_simple - initial_balance
                        profit_kelly = mean_balance_kelly - initial_balance

                        dist_col.append(dist)
                        league_col.append(league)
                        rps_col.append(round(rps_avg,4))
                        brier_col.append(round(brier_score,4))
                        acc_col.append(round(accuracy,4))
                        simple_col.append(round(profit_simple))
                        kelly_col.append(round(profit_kelly))

                        team_names = np.array(points_df['Team'].T)
                        team_points = np.array(points_df['Simulated Average Points'].T).astype(float)

                        # print(team_points)
                        team_points = np.round(team_points, 2)
                        points_list = [f' ({point})' for point in team_points ]
                        team_points = np.array(points_list)
                        team_name_and_points = team_names+team_points
                        # print(team_name_and_points)

                        #export images
                        if export_images == True:
                            plot_rps(range(0,len(rps_all),1),rps_all,f"Experiment ID {experiment_id}", "Match Index", "RPS", f"{experiment}\\RPS",f"{experiment_id}.png")
                            if league =='ENG1':
                                league_bal = 3800
                            else:
                                league_bal = 3060
                            plot_balances_simulated(league_bal,range(1,len(simulated_simple_balance[0])+1,1),simulated_simple_balance, f"Experiment ID {experiment_id}",  "Match Index", "Balance Euro (€)", f"{experiment}\\Naive",f"{experiment_id}.png")
                            plot_balances_simulated(league_bal,range(1,len(simulated_kelly_balance[0])+1,1),simulated_kelly_balance, f"Experiment ID {experiment_id}",  "Match Index", "Balance Euro (€)", f"{experiment}\\Kelly",f"{experiment_id}.png")
                            #Convergence check:
                            for i in range(0, len(team_names), 1):
                                if team_names[i] == 'Liverpool' or team_names[i] == 'Bayern Munich':
                                    plot_points(range(0,len(all_points[team_names[i]]),1),all_points[team_names[0]],f"Experiment ID: {experiment_id}: {team_names[i]}", "Simulated Tournament Iteration", "Points", f"Convergence\\{experiment}",f"{experiment_id}.png")

                        if league == 'ENG1':
                            data_eng_tournament[f'{dist} {league} {season}'] = team_name_and_points
                            # i+=1
                        else:
                            data_ger_tournament[f'{dist} {league} {season}']=team_name_and_points.tolist()
                            # j+=1

    data_metrics = {
        'Distribution': dist_col,
        'Predicted League': league_col,
        'RPS_avg': rps_col,
        'Brier': brier_col,
        'Acc': acc_col,
        'Profit_simple':simple_col,
        'Profit_kelly':kelly_col
    }


    df_metrics = pd.DataFrame(data_metrics)
    df_eng_tournaments = pd.DataFrame(data_eng_tournament)
    df_ger_tournaments = pd.DataFrame(data_ger_tournament)

    if export_files == True:
        if not os.path.exists(f"Export/Files"):
            os.makedirs(f"Export/Files")
        df_metrics.to_csv(f"Export/{experiment}_metrics.csv")
        df_ger_tournaments.to_csv(f"Export/{experiment}_ger_tournament.csv")
        df_eng_tournaments.to_csv(f"Export/{experiment}_eng_tournament.csv")


C:\Users\steve\ict_3913_stevegalea\models\evaluate\FYP
10 Window Experiments
Exporting Experiment 1
Exporting Experiment 2
Exporting Experiment 3
Exporting Experiment 4
Exporting Experiment 5
Exporting Experiment 6
Exporting Experiment 7
Exporting Experiment 8
15 Window Experiments
Exporting Experiment 1
Exporting Experiment 2
Exporting Experiment 3
Exporting Experiment 4
Exporting Experiment 5
Exporting Experiment 6
Exporting Experiment 7
Exporting Experiment 8
20 Window Experiments
Exporting Experiment 1
Exporting Experiment 2
Exporting Experiment 3
Exporting Experiment 4
Exporting Experiment 5
Exporting Experiment 6
Exporting Experiment 7
Exporting Experiment 8
ELO Model Experiments
Exporting Experiment 1
Exporting Experiment 2
Exporting Experiment 3
Exporting Experiment 4
Exporting Experiment 5
Exporting Experiment 6
Exporting Experiment 7
Exporting Experiment 8
